In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from MonteCarlo import MonteCarlo
import random 

we need to check following arbitrage free conditions: For strike $K_i$, time to maturity $\tau$
$$ \left(S_{0}-K_{i} e^{-r \tau}\right)^{+}<c_{0}<S_{0} $$
$$ \frac{C_{i, j}-C_{i-1, j}}{K_{i}-K_{i-1}} \in [-1,0] $$
$$ C_{i-1, j}-C_{i, j} \geqslant \frac{K_{i}-K_{i-1}}{K_{i+1}-K_{i}}\left(C_{i, j}-C_{i+1, j}\right) $$
$$ C_{i, j+1}-C_{i, j} \geq 0 $$

In [ ]:
# def arbitrage_check(data):
#     cols = ['moneyness', 'tau', 'r', 'vol', 'C']
#     data = pd.DataFrame(data, columns=cols)
#     data['K'] = 1 / data['moneyness']
#     data = data.sort_values(by='K')
#     data['K-1'] = data['K'].shift(1)
#     data['K+1'] = data['K'].shift(-1)
#     data['C-1'] = data['C'].shift(1)
#     data['C+1'] = data['C'].shift(-1)
#     data = data.iloc[1:-1]
#     # check 1
#     data['check1'] = np.maximum(1 - data['K'] * np.exp(- data['r'] * data['tau']), 0)
#     data = data.loc[(data['C'] < 1) & (data['C'] > data['check1'])] 
#     # check 2 
#     data['check2'] = (data['C'] - data['C-1']) / (data['K'] - data['K-1'])
#     data = data.loc[(data['check2'] >= -1) & (data['check2'] <= 0)]
#     # check 3
#     data['check3'] = (data['C'] - data['C-1']) / (data['C+1'] - data['C']) - (data['K'] - data['K-1']) / (data['K+1'] - data['K'])
#     data = data.loc[data['check3'] >= 0]
#     print(data)

# data = pd.read_csv('test.csv')
# data.drop('Unnamed: 0', inplace=True, axis=1)
# data = data[data['tau']==data['tau'][0]]
# arbitrage_check(data.values.tolist())



<font size=20>European call + GBM + Closed-form formula</font>

In [6]:
model = 'GBM' 
Type = 'European' 
option = 'call' 

American = False if Type == 'European' else True
optionType = 'c' if option == 'call' else 'p'

cols = ['moneyness', 'tau', 'r', 'vol', 'option price']
data = []

while len(data) < 10000:
    for i in tqdm(range(5000)):
        vol = random.uniform(0.02, 0.9)
        r = random.uniform(0.03, 0.08)
        tau = random.uniform(0.3, 0.95)
        moneyness = random.uniform(0.7, 1.3)
        K = 1 / moneyness
        MC = MonteCarlo(model='GBM', S0=1, K=K, T=tau, r=r, q=0, v=vol, method='formula')
        MC.generate_S(1000,10)
        option_price = MC.pricer(optionType=optionType, American=American)
        if option_price > 0.01:
            data.append([moneyness, tau, r, vol, option_price])

data = pd.DataFrame(data, columns=cols)
data.to_csv(f'data_{model}_{Type}_{option}.csv', index=False)



100%|██████████| 5000/5000 [00:02<00:00, 1677.42it/s]


       moneyness       tau         r       vol  option price
3163    0.842777  0.533024  0.036294  0.187983      0.010036
2028    0.921883  0.316636  0.077697  0.133479      0.010038
12830   0.796456  0.483397  0.051270  0.240633      0.010107
2070    0.846692  0.784535  0.039505  0.144355      0.010158
4177    0.954340  0.387281  0.072730  0.071875      0.010169
...          ...       ...       ...       ...           ...
10493   1.254532  0.906340  0.076512  0.888865      0.430089
412     1.292918  0.947030  0.030125  0.887099      0.431446
10832   1.281616  0.918517  0.070999  0.876070      0.434199
1691    1.248467  0.940501  0.076109  0.893988      0.435330
12361   1.291263  0.949613  0.056372  0.893441      0.441792

[13832 rows x 5 columns]


In [5]:
data.sort_values(by='option price')

,moneyness,tau,r,vol,option price
9268,0.721101,0.307023,0.078133,0.398930,0.010013
4986,0.891375,0.572469,0.073761,0.114388,0.010030
7260,0.837894,0.878324,0.038886,0.140775,0.010066
3441,0.748879,0.352308,0.063310,0.340839,0.010068
4235,0.901987,0.700607,0.056933,0.095568,0.010080
...,...,...,...,...,...
823,1.266673,0.890929,0.072211,0.865899,0.423184
12907,1.295607,0.848782,0.070777,0.864142,0.423604
711,1.295815,0.913508,0.058887,0.841988,0.423890
8591,1.262412,0.936385,0.079033,0.858610,0.429410


In [4]:
data.describe()

,moneyness,tau,r,vol,option price
count,10537.000000,10537.000000,10537.000000,10537.000000,10537.000000
mean,1.063708,0.643051,0.055380,0.542235,0.218922
std,0.157691,0.184528,0.014396,0.237476,0.071644
min,0.700032,0.300022,0.030001,0.020041,0.100056
25%,0.949484,0.488232,0.043024,0.362569,0.160809
50%,1.087720,0.650307,0.055603,0.578912,0.215439
75%,1.194842,0.802350,0.067763,0.740467,0.267869
max,1.299951,0.949958,0.079999,0.899886,0.430429


In [31]:
data = pd.read_csv(f'data_{model}_{Type}_{option}.csv')

x = data[['moneyness', 'tau', 'r', 'vol']]
y = data['option price']

scoring = {
    'r2':'r2', 
    'MSE': make_scorer(mean_squared_error),
    'MAPE': make_scorer(mean_absolute_percentage_error),
    'MAE': make_scorer(mean_absolute_error),    
    }
param_grid = {
    'activation': ['relu'],
    'solver': ['adam'],
    'hidden_layer_sizes': 
        [(2 ** i, 2 ** i) for i in np.arange(3, 7)] + 
        [(2 ** i, 2 ** i, 2 ** i) for i in np.arange(3, 7)] + 
        [(2 ** i, 2 ** i, 2 ** i, 2 ** i) for i in np.arange(3, 7)],
    'max_iter': [500],
    'learning_rate_init': np.arange(0.005, 0.005+0.001, 0.001),
}
MLP = MLPRegressor(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', batch_size=1024, learning_rate_init=0.02, max_iter=500, tol=1e-10, shuffle=True, verbose=False, validation_fraction=0.1)
search = GridSearchCV(MLP, param_grid=param_grid, cv=5, scoring = scoring, refit='MSE', return_train_score=True, verbose=4, n_jobs=3)
search.fit(x,y)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV 3/5] END activation=logistic, hidden_layer_sizes=(8, 8), learning_rate_init=0.005, max_iter=500, solver=adam; MAE: (train=0.013, test=0.013) MAPE: (train=0.071, test=0.070) MSE: (train=0.000, test=0.000) r2: (train=0.942, test=0.943) total time=   1.6s
[CV 1/5] END activation=logistic, hidden_layer_sizes=(8, 8), learning_rate_init=0.005, max_iter=500, solver=adam; MAE: (train=0.013, test=0.013) MAPE: (train=0.068, test=0.068) MSE: (train=0.000, test=0.000) r2: (train=0.942, test=0.942) total time=   3.8s


/Users/lusihan/Desktop/env/TBIC/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 2/5] END activation=logistic, hidden_layer_sizes=(8, 8), learning_rate_init=0.005, max_iter=500, solver=adam; MAE: (train=0.013, test=0.013) MAPE: (train=0.068, test=0.068) MSE: (train=0.000, test=0.000) r2: (train=0.945, test=0.945) total time=   5.1s
[CV 4/5] END activation=logistic, hidden_layer_sizes=(8, 8), learning_rate_init=0.005, max_iter=500, solver=adam; MAE: (train=0.011, test=0.011) MAPE: (train=0.059, test=0.058) MSE: (train=0.000, test=0.000) r2: (train=0.947, test=0.947) total time=   4.7s
[CV 1/5] END activation=logistic, hidden_layer_sizes=(16, 16), learning_rate_init=0.005, max_iter=500, solver=adam; MAE: (train=0.013, test=0.013) MAPE: (train=0.067, test=0.067) MSE: (train=0.000, test=0.000) r2: (train=0.941, test=0.941) total time=   2.4s


/Users/lusihan/Desktop/env/TBIC/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 5/5] END activation=logistic, hidden_layer_sizes=(8, 8), learning_rate_init=0.005, max_iter=500, solver=adam; MAE: (train=0.011, test=0.012) MAPE: (train=0.061, test=0.064) MSE: (train=0.000, test=0.000) r2: (train=0.954, test=0.953) total time=   5.1s
[CV 3/5] END activation=logistic, hidden_layer_sizes=(16, 16), learning_rate_init=0.005, max_iter=500, solver=adam; MAE: (train=0.013, test=0.013) MAPE: (train=0.066, test=0.065) MSE: (train=0.000, test=0.000) r2: (train=0.941, test=0.943) total time=   1.5s
[CV 2/5] END activation=logistic, hidden_layer_sizes=(16, 16), learning_rate_init=0.005, max_iter=500, solver=adam; MAE: (train=0.014, test=0.013) MAPE: (train=0.074, test=0.073) MSE: (train=0.000, test=0.000) r2: (train=0.942, test=0.943) total time=   3.8s
[CV 4/5] END activation=logistic, hidden_layer_sizes=(16, 16), learning_rate_init=0.005, max_iter=500, solver=adam; MAE: (train=0.013, test=0.013) MAPE: (train=0.068, test=0.067) MSE: (train=0.000, test=0.000) r2: (train=0.94

...

/Users/lusihan/Desktop/env/TBIC/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 2/5] END activation=tanh, hidden_layer_sizes=(256, 256, 256), learning_rate_init=0.005, max_iter=500, solver=adam; MAE: (train=0.004, test=0.004) MAPE: (train=0.020, test=0.021) MSE: (train=0.000, test=0.000) r2: (train=0.996, test=0.995) total time= 5.8min


/Users/lusihan/Desktop/env/TBIC/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 3/5] END activation=tanh, hidden_layer_sizes=(256, 256, 256), learning_rate_init=0.005, max_iter=500, solver=adam; MAE: (train=0.003, test=0.003) MAPE: (train=0.018, test=0.018) MSE: (train=0.000, test=0.000) r2: (train=0.996, test=0.996) total time= 5.8min
[CV 1/5] END activation=tanh, hidden_layer_sizes=(8, 8, 8, 8), learning_rate_init=0.005, max_iter=500, solver=adam; MAE: (train=0.002, test=0.002) MAPE: (train=0.013, test=0.013) MSE: (train=0.000, test=0.000) r2: (train=0.998, test=0.998) total time=   6.7s
[CV 2/5] END activation=tanh, hidden_layer_sizes=(8, 8, 8, 8), learning_rate_init=0.005, max_iter=500, solver=adam; MAE: (train=0.005, test=0.005) MAPE: (train=0.026, test=0.027) MSE: (train=0.000, test=0.000) r2: (train=0.992, test=0.991) total time=   4.4s
[CV 3/5] END activation=tanh, hidden_layer_sizes=(8, 8, 8, 8), learning_rate_init=0.005, max_iter=500, solver=adam; MAE: (train=0.004, test=0.004) MAPE: (train=0.024, test=0.023) MSE: (train=0.000, test=0.000) r2: (train

KeyboardInterrupt: 

In [6]:
search.cv_results_

{'mean_fit_time': array([ 3.11932492,  3.09468063,  3.88210638,  6.33453067, 23.2832106 ,
        42.13535802,  3.27311118,  3.03968771,  3.28887971,  4.35198983,
         4.96050763, 14.33884764,  3.83869608,  3.09853538,  4.58223295,
         4.55477913,  7.70102318, 21.48735515,  1.5271457 ,  2.92117492,
         6.75823243, 17.92416128, 41.67616073, 68.18525942,  1.9107457 ,
         4.10255361,  9.1273183 , 23.72158734, 24.43632499, 16.86052815,
         2.88715235,  2.73964373,  6.57039841, 10.98808996, 21.01670464,
        25.69916821,  1.92987569,  1.78623025,  2.22022096,  2.36539666,
         6.29528904, 19.12512668,  0.66204866,  1.42757575,  2.69754926,
         3.73380502,  5.88949633, 35.25627947,  1.79704698,  1.35481675,
         2.29301866,  3.4771502 , 10.39745498, 26.29678742]),
 'std_fit_time': array([1.73071106, 1.28591816, 0.37357458, 0.70045591, 6.30986761,
        6.24130006, 1.5577388 , 1.10930243, 2.34424124, 2.3118937 ,
        1.00371797, 1.95959741, 0.55166

In [7]:
search.best_params_

{'activation': 'logistic',
 'hidden_layer_sizes': (256, 256, 256, 256),
 'learning_rate_init': 0.02,
 'max_iter': 500,
 'solver': 'adam'}

In [10]:
search.best_score_

0.005157212105318282

In [19]:
data = pd.read_csv(f'data_{model}_{Type}_{option}.csv')
print(data.shape)

x = data[['moneyness', 'tau', 'r', 'vol']]
y = data['option price']

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=None, shuffle=True)

# StdScaler = StandardScaler()
# x_train = StdScaler.fit_transform(x_train)

# StdScaler = StandardScaler()
# x_test = StdScaler.fit_transform(x_test)

MLP = MLPRegressor(hidden_layer_sizes=(32, 32, 32, 32), activation='relu', solver='adam', batch_size=1024, learning_rate_init=0.005, max_iter=500, tol=1e-10, shuffle=True, verbose=False, validation_fraction=0.1)
# MLP.fit(x_train, y_train)
# score = MLP.score(x_test, y_test)
# print(f'Type: {Type}, option: {option}, Model: {model}, ANN r2 = {score}')

# r2
print('r2:')
print(cross_val_score(MLP, x, y, scoring='r2', cv=3))
# MSE
print('MSE:')
print(cross_val_score(MLP, x, y, scoring='neg_mean_squared_error', cv=3))
# MAE
print('MAE:')
print(cross_val_score(MLP, x, y, scoring='neg_mean_absolute_error', cv=3))
# MAPE
print('MAPE:')
print(cross_val_score(MLP, x, y, scoring='neg_mean_absolute_error', cv=3))

(13832, 5)
r2:
[0.99654136 0.99478643 0.99668479]
MSE:
[-2.37234072e-05 -1.14050635e-04 -2.29888087e-05]
MAE:
[-0.01280249 -0.00301259 -0.00295126]
MAPE:


array([-0.00473464, -0.00349429, -0.0047904 ])